# Movies and Books Assistant with Weaviate Query Agent
**Demo for the Reimanining Data Workflows Webinar - April 29th 2025**

Example code for how we can build an agent which can answer quuestions about books and movies, using the [Weaviate Query Agent](https://weaviate.io/developers/agents?utm_source=webinar&utm_campaign=agents&utm_content=reimagining-data-workflows).

> 📚 You can read and learn more about this service in our ["Introducing the Weaviate Query Agent"](https://weaviate.io/blog/query-agent?utm_source=webinar&utm_campaign=agents&utm_content=reimagining-data-workflows) blog.

To get started, we've prepared a few open datasets, available on Hugging Face. The first step will be walking through how to populate your Weaviate Cloud collections.

- [**Movies:**](https://huggingface.co/datasets/weaviate/agents/viewer/personalization-agent-movies) A dataset that lists movies, their descriptions, ratings etc.
- [**Books:**](https://huggingface.co/datasets/weaviate/agents/viewer/query-agent-books) A dataset that lists books, their authors, descriptions and genres.


>[Build A Weaviate Query Agent - The E-Commerce Assistant](#scrollTo=iGfss7TuSM_n)

>>[Setting Up Weaviate & Importing Data](#scrollTo=V7eGoTMgV597)

>>>[Prepare the Collections](#scrollTo=XqbFu9xHMzEU)

>>[Set Up the Query Agent](#scrollTo=4H7h-onziDEA)

>>[Run the Query Agent](#scrollTo=XnJiTt5LiMp-)

>>>[Ask a Question](#scrollTo=XnJiTt5LiMp-)

>>>[Ask a follow up question](#scrollTo=RqsUGStI-RC_)

>>>[Search over multiple collections](#scrollTo=sl78PHYmBe2s)

>>>[Changing the System Prompt](#scrollTo=Hm32UutOC63l)

>>>[Try More Questions](#scrollTo=70lwuxf1F38d)



## 1. Setting Up Weaviate & Importing Data

To use the Weaviate Query Agent, first, create a [Weaviate Cloud](https://weaviate.io/deployment/serverless?utm_source=webinar&utm_campaign=agents&utm_content=reimagining-data-workflows) account👇
1. [Create Serverless Weaviate Cloud account](https://weaviate.io/deployment/serverless?utm_source=webinar&utm_campaign=agents&utm_content=reimagining-data-workflows) and setup a free [Sandbox](https://weaviate.io/developers/wcs/manage-clusters/create#sandbox-clusters?utm_source=webinar&utm_campaign=agents&utm_content=reimagining-data-workflows)
2. Go to 'Embedding' and enable it, by default, this will make it so that we use `Snowflake/snowflake-arctic-embed-l-v2.0` as the embedding model
3. Take note of the `WEAVIATE_URL` and `WEAVIATE_API_KEY` to connect to your cluster below

> Info: We recommend using [Weaviate Embeddings](https://weaviate.io/developers/weaviate/model-providers/weaviate?utm_source=webinar&utm_campaign=agents&utm_content=reimagining-data-workflows) so you do not have to provide any extra keys for external embedding providers.

In [1]:
!pip install weaviate-client[agents] datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.3/442.3 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

In [2]:
import os
from getpass import getpass

if "WEAVIATE_API_KEY" not in os.environ:
  os.environ["WEAVIATE_API_KEY"] = getpass("Weaviate API Key")
if "WEAVIATE_URL" not in os.environ:
  os.environ["WEAVIATE_URL"] = getpass("Weaviate URL")

Weaviate API Key··········
Weaviate URL··········


In [3]:
import weaviate
from weaviate.auth import Auth

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.environ.get("WEAVIATE_URL"),
    auth_credentials=Auth.api_key(os.environ.get("WEAVIATE_API_KEY")),
)

### Prepare the Collections

In the following code blocks, we are pulling our demo datasets from Hugging Face and writing them to new collections in our Weaviate Serverless cluster.

> ❗️ The `QueryAgent` uses the descriptions of collections and properties to decide which ones to use when solving queries, and to access more information about properties. You can experiment with changing these descriptions, providing more detail, and more. It's good practice to provide property descriptions too.

In [14]:
from weaviate.classes.config import Configure, Property, DataType

# client.collections.delete("Movies")
# client.collections.delete("Books")
client.collections.create(
    "Books",
    description="A dataset that lists books, their author, description and genres",
    vectorizer_config=Configure.Vectorizer.text2vec_weaviate(),
    properties=[
        Property(name="title", data_type=DataType.TEXT, description="title of the book"),
        Property(name="author", data_type=DataType.TEXT, description="author of the book"),
        Property(name="description", data_type=DataType.TEXT, description="description of the book"),
        Property(name="genres", data_type=DataType.TEXT_ARRAY, description="genres of the book"),
      ]
)

client.collections.create(
    "Movies",
    description="A dataset that lists movies, their ratings, original language etc..",
    vectorizer_config=Configure.Vectorizer.text2vec_weaviate(),
    properties=[
        Property(
            name="title",
            data_type=DataType.TEXT,
            description="The title of the movie",
        ),
        Property(
            name="release_date",
            data_type=DataType.TEXT,
            description="The release date of the movie",
        ),
        Property(
            name="overview",
            data_type=DataType.TEXT,
            description="Short description of the movie",
        ),
        Property(
            name="genres",
            data_type=DataType.TEXT_ARRAY,
            description="The genres of the movie",
        ),
        Property(
            name="vote_average",
            data_type=DataType.NUMBER,
            description="vote average of the movie",
        ),
        Property(
            name="vote_count",
            data_type=DataType.INT,
            description="vote count of the movie",
        ),
        Property(
            name="popularity",
            data_type=DataType.NUMBER,
            description="popularity of the movie",
        ),
        Property(
            name="poster_url",
            data_type=DataType.TEXT,
            description="poster path of the movie",
            skip_vectorization=True,
        ),
        Property(
            name="original_language",
            data_type=DataType.TEXT,
            description="Code of the language of the movie",
            skip_vectorization=True,
        ),
    ]
)


In [15]:
from datasets import load_dataset

# movies_dataset = load_dataset("weaviate/agents", "personalization-agent-movies", split="train", streaming=True)
# books_dataset = load_dataset("weaviate/agents", "query-agent-books", split="train", streaming=True)

movies_collection = client.collections.get("Movies")
books_collection = client.collections.get("Books")

# with movies_collection.batch.dynamic() as batch:
#     for item in movies_dataset:
#         batch.add_object(properties=item["properties"])

# with books_collection.batch.dynamic() as batch:
#     for item in books_dataset:
#         batch.add_object(properties=item["properties"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.81k [00:00<?, ?B/s]

## 2. Set Up the Query Agent

Let's start with a simple agent. Here, we're creating an `agent` that has access to our `Books` & `Movies` datasets.

In [16]:
from weaviate.agents.query import QueryAgent

agent = QueryAgent(
    client=client, collections=["Books", "Movies"],
)

## 3. Run the Query Agent
The `QueryAgent` will determine wither a given query is a regular searcg query (vector search), or whether it requires aggregations, or both.

In [17]:
from weaviate.agents.utils import print_query_agent_response

response = agent.run("What are some good fantasy films that involve elves?")
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ What are some good fantasy films that involve elves?                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ Here are some notable fantasy films involving elves that you might enjoy:                                       │
│                                                                                                                 │
│ 1. **Throne of Elves (2016)**: Set in the mythical Elven world of Altera, this film features an epic battle     │
│ between Princess Liya and the Dark Elf Elena, who possesses a powerful jewel. (Source 1)                        │
│                                                                                                                 │
│ 2. **David and the Elves (2021)**: This family-oriented movie centers around David and an elf named Albert      │
│ embarking on a journey to restore the meaning of Christmas. (Source 2)                                          │
│                                                                                                                 │
│ 3. **Dragon Nest: Warriors' Dawn (2014)**: In this story, humans and elves join forces to fight against beasts  │
│ loyal to a Black Dragon as they try to reach the dragon’s lair. (Source 3)                                      │
│                                                                                                                 │
│ 4. **Onward (2020)**: This animated film follows two elf brothers on a quest to find out if there is still      │
│ magic left in their world. (Source 5)                                                                           │
│                                                                                                                 │
│ 5. **A Boy Called Christmas (2021)**: An extraordinary adventure of a young boy named Nikolas, who searches for │
│ the village of the elves, Elfhelm. (Source 8)                                                                   │
│                                                                                                                 │
│ 6. **Ella Enchanted (2004)**: Ella, who has the gift of obedience, embarks on a journey where she encounters    │
│ elves among other magical creatures. (Source 10)                                                                │
│                                                                                                                 │
│ 7. **Elf (2003)**: A comedy about Buddy, a human raised by elves in the North Pole, who travels to New York to  │
│ find his real father. (Source 11)                                                                               │
│                                                                                                                 │
│ 8. **Bright: Samurai Soul (2021)**: In this animated film, a Ronin and an orc escort a young elf girl through a │
│ fantasy world. (Source 20)                                                                                      │
│                                                                                                                 │
│ Each of these films offers a unique exploration of fantasy worlds and the intriguing roles that elves play      │
│ within them.                                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['fantasy films elves'],                                                                            │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='Movies'                                                                                         │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='d6b48e74-b6f9-4fb0-96c9-109e8b431c83' collection='Movies'                                         │
│  - object_id='58a537df-670e-4d86-9c56-4a6dc9cb3317' collection='Movies'                                         │
│  - object_id='96982f3c-9847-4b76-a63d-9f3bb859fbf2' collection='Movies'                                         │
│  - object_id='85ffcd5f-9ce0-4f99-9f79-08d951bd8c4f' collection='Movies'                                         │
│  - object_id='85b1a40a-dd84-4429-92cf-c64d0a6fa0bf' collection='Movies'                                         │
│  - object_id='d695b7af-b9c6-4cf2-8f7f-13e3e8b74de8' collection='Movies'                                         │
│  - object_id='9d296c7e-0556-4213-9630-42952e32fda2' collection='Movies'                                         │
│  - object_id='6ad13de6-7908-4aa2-8234-e55bae0f8692' collection='Movies'                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics   
┌────────────────┬──────┐
│ LLM Requests:  │ 3    │
│ Input Tokens:  │ 6406 │
│ Output Tokens: │ 502  │
│ Total Tokens:  │ 6908 │
└────────────────┴──────┘

Total Time Taken: 12.50s

### Ask a follow up question

The agent can also be provided with additional context. For example, we can provide the previous response as context and get a `new_response`

In [18]:
new_response = agent.run("And what are some books with a similar vibe to these films?", context=response)
new_response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ And what are some books with a similar vibe to these films?                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ Here are some books with a similar fantasy vibe to the films featuring elves:                                   │
│                                                                                                                 │
│ 1. **Lark (Lark, #1)** by Erica Cope - This book involves Mia Carrington, who discovers she is the daughter of  │
│ the King of the Light Elves. An ominous prophecy predicts that she will unleash the Dark Elves, setting a       │
│ fantasy adventure in motion with elements of magic and intrigue.                                                │
│                                                                                                                 │
│ 2. **The Dark Elf Trilogy Collector's Edition (Forgotten Realms: Dark Elf Trilogy, #1-3; Legend of Drizzt,      │
│ #1-3)** by R.A. Salvatore - A set of books revolving around elves in the Forgotten Realms universe, combining   │
│ epic fantasy with adventure.                                                                                    │
│                                                                                                                 │
│ 3. **The Elf Queen of Shannara** by Terry Brooks - Part of the Shannara series, this book features a quest      │
│ involving the search for the missing Elves and contains themes of epic fantasy and magic.                       │
│                                                                                                                 │
│ 4. **Blood of Elves (The Witcher, #1)** by Andrzej Sapkowski - In this book, the uneasy peace among humans,     │
│ dwarves, gnomes, and elves breaks down leading to conflict, aligning with the fantasy settings of films that    │
│ include elves.                                                                                                  │
│                                                                                                                 │
│ These books share elements of epic fantasy, magic, and adventure, similar to the themes found in films          │
│ featuring elves.                                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['fantasy books with elves'],                                                                       │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='Books'                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='5678eb39-b17e-4ccb-b8c0-a00968ce06c4' collection='Books'                                          │
│  - object_id='37c4b9a1-f19b-412e-b992-5a9d579f3309' collection='Books'                                          │
│  - object_id='cb11b35b-7c90-415d-b41a-bc24d113e1f5' collection='Books'                                          │
│  - object_id='277a86af-4872-479b-adc9-ce4e77bee2d8' collection='Books'                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics    
┌────────────────┬───────┐
│ LLM Requests:  │ 4     │
│ Input Tokens:  │ 10454 │
│ Output Tokens: │ 532   │
│ Total Tokens:  │ 10986 │
└────────────────┴───────┘

Total Time Taken: 15.21s

Now let's try a question that sholud require an aggregation. Let's see which author has the most books in our collection.

In [19]:
response = agent.run("Which author has the most books listed in our collection?")
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ Which author has the most books listed in our collection?                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ The author with the most books listed in the collection is Stephen King, with 57 books.                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔭 No Searches Run                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 📊 Aggregations Run 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ AggregationResultWithCollection(                                                                                │
│     search_query=None,                                                                                          │
│     groupby_property=None,                                                                                      │
│     aggregations=[                                                                                              │
│         TextPropertyAggregation(                                                                                │
│             property_name='author',                                                                             │
│             metrics=<TextMetrics.TOP_OCCURRENCES: 'TOP_OCCURRENCES'>,                                           │
│             top_occurrences_limit=1                                                                             │
│         )                                                                                                       │
│     ],                                                                                                          │
│     filters=[],                                                                                                 │
│     collection='Books'                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics   
┌────────────────┬──────┐
│ LLM Requests:  │ 3    │
│ Input Tokens:  │ 3389 │
│ Output Tokens: │ 146  │
│ Total Tokens:  │ 3535 │
└────────────────┴──────┘

Total Time Taken: 4.77s

In [20]:
response = agent.run("And are there any films based on this author's books?", context=response)
response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ And are there any films based on this author's books?                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ Yes, there are several films based on Stephen King's books. Here are a few examples:                            │
│                                                                                                                 │
│ 1. **Creepshow (1982)**: Directed by George A. Romero and with a screenplay by Stephen King, this film brings   │
│ five tales of terror to the screen. It's inspired by the E.C. comics of the 1950s and is a unique collaboration │
│ between King and Romero, with a mix of horror and comedy.                                                       │
│                                                                                                                 │
│ 2. **Big Driver (2014)**: This film is based on a novella by Stephen King. It tells the story of a famous       │
│ mystery writer who sets out for revenge after a brutal attack, and it's a crime, mystery, and thriller film.    │
│                                                                                                                 │
│ 3. **Sometimes They Come Back (1991)**: Adapted from a Stephen King short story, this horror film features a    │
│ man and his family who return to his hometown, facing harassment from teenagers who died when he was a child.   │
│                                                                                                                 │
│ 4. **Cat's Eye (1985)**: A horror-thriller anthology film that adapts three of King's short stories, linked by  │
│ a stray cat that roams from one tale to the next. Each story presents its own chilling narrative with elements  │
│ of supernatural intrigue.                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['films adapted from Stephen King books'],                                                          │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='Movies'                                                                                         │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='32d5c53f-0768-4cbf-bfc0-18410f72ba29' collection='Movies'                                         │
│  - object_id='20b016ae-277e-4b03-a89a-202d6e6917df' collection='Movies'                                         │
│  - object_id='88ddd9d3-f326-41a2-9f8a-2ae199f05c4c' collection='Movies'                                         │
│  - object_id='3f0db086-c0d2-4ac6-aa2c-ed1870d4e341' collection='Movies'                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics   
┌────────────────┬──────┐
│ LLM Requests:  │ 4    │
│ Input Tokens:  │ 8163 │
│ Output Tokens: │ 483  │
│ Total Tokens:  │ 8646 │
└────────────────┴──────┘

Total Time Taken: 12.98s

### Search over multiple collections

In some cases, we need to combine the results of searches across multiple collections.

In [23]:
response = agent.run("I'm interested in historical fiction books, can you recommend any good ones?"
                     "Are there any films based on historical fiction? And on average, what's the original language that they were filmed in?")

response.display()

╭─────────────────────────────────────────────── 🔍 Original Query ───────────────────────────────────────────────╮
│                                                                                                                 │
│ I'm interested in historical fiction books, can you recommend any good ones?Are there any films based on        │
│ historical fiction? And on average, what's the original language that they were filmed in?                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Final Answer ────────────────────────────────────────────────╮
│                                                                                                                 │
│ If you're interested in historical fiction books, here are some notable recommendations:                        │
│                                                                                                                 │
│ 1. **"Castles, Customs, and Kings"** by Debra Brown: This is a compilation of essays offering a rich historical │
│ backdrop, exploring Great Britain's history from Roman times to the early 20th century.                         │
│                                                                                                                 │
│ 2. **"Post Captain"** by Patrick O'Brian: Part of the Aubrey & Maturin series, this book is acclaimed for its   │
│ detailed depiction of naval life during the Napoleonic Wars.                                                    │
│                                                                                                                 │
│ 3. **"People of the Book"** by Geraldine Brooks: This novel traces the journey of a rare manuscript, delving    │
│ into historical events spanning several centuries.                                                              │
│                                                                                                                 │
│ 4. **"The Shadow of the Wind"**, **"The Angel's Game"**, and **"The Prisoner of Heaven"** by Carlos Ruiz Zafón: │
│ Set in a post-war Barcelona, these novels intertwine mystery and history within the Cemetery of Forgotten Books │
│ series.                                                                                                         │
│                                                                                                                 │
│ Regarding films based on historical fiction, some notable examples include:                                     │
│                                                                                                                 │
│ - **"Suffragette"**: A drama about the early feminist movement.                                                 │
│ - **"300"**: A film loosely based on the Battle of Thermopylae.                                                 │
│ - **"Casualties of War"**: Depicting events from the Vietnam War.                                               │
│ - **"The Warlords"**: Set amidst the Qing Dynasty, focusing on internal political strife.                       │
│                                                                                                                 │
│ Films based on historical fiction are predominantly filmed in English, with it being the original language for  │
│ a vast majority of them.                                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 1/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['historical fiction books'],                                                                       │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='Books'                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── 🔭 Searches Executed 2/2 ────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollection(                                                                                      │
│     queries=['films based on historical fiction'],                                                              │
│     filters=[[]],                                                                                               │
│     filter_operators='AND',                                                                                     │
│     collection='Movies'                                                                                         │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 📊 Aggregations Run 1/1 ────────────────────────────────────────────╮
│                                                                                                                 │
│ AggregationResultWithCollection(                                                                                │
│     search_query='historical fiction',                                                                          │
│     groupby_property='original_language',                                                                       │
│     aggregations=[                                                                                              │
│         IntegerPropertyAggregation(                                                                             │
│             property_name='original_language',                                                                  │
│             metrics=<NumericMetrics.COUNT: 'COUNT'>                                                             │
│         )                                                                                                       │
│     ],                                                                                                          │
│     filters=[],                                                                                                 │
│     collection='Movies'                                                                                         │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='b23a88d8-5d49-4871-a69e-31e11761a14c' collection='Books'                                          │
│  - object_id='7789f34a-2b9f-4a9f-82e0-26575faac318' collection='Books'                                          │
│  - object_id='caff6c2d-cef6-4ce9-98f3-48cf57d66b31' collection='Books'                                          │
│  - object_id='81023cdc-1332-4cab-973c-c28c7ccfae73' collection='Books'                                          │
│  - object_id='428f1342-f2c2-4a9a-90af-16a8d46df455' collection='Books'                                          │
│  - object_id='17f11071-e357-4e7b-be75-45f9a140e8fa' collection='Books'                                          │
│  - object_id='600fa2ea-b0a2-4d87-98bc-d0d27f9f5ad2' collection='Movies'                                         │
│  - object_id='3faf3d17-b4d1-41c0-bd0d-4a8e37d3624a' collection='Movies'                                         │
│  - object_id='19fe42f2-0c41-40dc-a781-03838fb46b8a' collection='Movies'                                         │
│  - object_id='212b015d-375f-4b11-9db3-af8b81ca32b1' collection='Movies'                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

   📊 Usage Statistics    
┌────────────────┬───────┐
│ LLM Requests:  │ 5     │
│ Input Tokens:  │ 11470 │
│ Output Tokens: │ 571   │
│ Total Tokens:  │ 12041 │
└────────────────┴───────┘

Total Time Taken: 12.82s

### Changing the System Prompt

In some cases, you may want to define a custom `system_prompt` for your agent. This can help you provide the agent with some default instructions as to how to behave. For example, let's create an agent that is designed to give short bullet point answers.


In [24]:
new_agent = QueryAgent(
    client=client, collections=["Books", "Movies"],
    system_prompt="You are a helpful movies and books assistant that always responds in short, bullet point answsers."
)

For example, this time lets ask something that is about weather!

In [25]:
response = new_agent.run("I'm interested in historical fiction books, can you recommend any good ones?"
                         "Are there any films based on historical fiction? And on average, what's the original language that they were filmed in?")
print(response.final_answer)

Here are some recommended historical fiction books:
- **Castles, Customs, and Kings**: A compilation of essays providing historical information from Roman Britain to early twentieth-century England. Authors share real-life stories and tidbits used in their historical novels.
- **Post Captain** by Patrick O'Brian: Part of the Aubrey & Maturin series, known as one of the best historical novels.
- **People of the Book** by Geraldine Brooks: Follows a rare manuscript through centuries, inspired by the true story of the Sarajevo Haggadah.
- **The Shadow of the Wind** by Carlos Ruiz Zafón: A mystery set in post-war Barcelona around a lost literary work.

Films based on historical fiction include:
- "Hamilton" (2020): A filmed version of the Broadway hit about American founding father Alexander Hamilton.
- "The Lost City of Z" (2017): The story of British explorer Percy Fawcett's adventures in the Amazon in the 1920s.
- "Suffragette" (2015): Based on true events about early feminist movement 